# Code Snippets
This notebook walks through the basics of how to use the AI21 models through the Bedrock API; giving a few code snippets you can use in your own work.

## Call AI21 Models via Amazon Bedrock API



In [ ]:
Following code snippet shows a call function for Bedrock Converse API

In [5]:
import os
import json
import boto3
import botocore
import logging
from botocore.client import Config
from botocore.exceptions import ClientError 

# Get response from bedrock via Converse API
# Support for converse API - https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html
def generate_conversation(bedrock_client,
                          model_id,
                          system_prompts,
                          messages):
    """
    Sends messages to a model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        system_prompts (JSON) : The system prompts for the model to use.
        messages (JSON) : The messages to send to the model.

    Returns:
        response (JSON): The conversation that the model generated.

    """

    logging.info("Generating message with model %s", model_id)

    # Inference parameters to use.
    temperature = 0.5
    top_p = 1

    # Base inference parameters to use.
    inference_config = {"temperature": temperature}
    
    # Additional inference parameters to use. For additional information - https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-jamba.html#api-inference-examples-a2i-jamba
    additional_model_fields = {"top_p": top_p}

    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )

    # Log token usage.
    token_usage = response['usage']
    logging.info("Input tokens: %s", token_usage['inputTokens'])
    logging.info("Output tokens: %s", token_usage['outputTokens'])
    logging.info("Total tokens: %s", token_usage['totalTokens'])
    logging.info("Stop reason: %s", response['stopReason'])

    return response


## Call Jamba 1.5
Following code snippet shows how to use AI21 Jamba 1.5 model using the converse API call function setup above

In [ ]:
#Jamba 1.5 is only supported in us-east-1 as of 08/20/24
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

def main():
    """
    Entrypoint for foundational model example.
    """

    logging.basicConfig(level=logging.INFO,
                        format="%(levelname)s: %(message)s")

    # For all model IDs, please visit https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html
    model_id = "ai21.jamba-1-5-large-v1:0" #Jamba 1.5 Large
    #model_id = "ai21.jamba-1-5-mini-v1:0" #Jamba 1.5 Mini

    # Setup the system prompts and messages to send to the model.
    system_prompts = [{"text": "You are an app that creates playlists for a radio station that plays rock and pop music."
                       "Only return song names and the artist."}]
    message_1 = {
        "role": "user",
        "content": [{"text": "Create a list of 3 pop songs."}]
    }
    message_2 = {
        "role": "user",
        "content": [{"text": "Make sure the songs are by artists from the United Kingdom."}]
    }
    messages = []

    try:

        bedrock_client = boto3.client(service_name='bedrock-runtime')

        # Start the conversation with the 1st message.
        messages.append(message_1)
        response = generate_conversation(
            bedrock_client, model_id, system_prompts, messages)

        # Add the response message to the conversation.
        output_message = response['output']['message']
        messages.append(output_message)

        # Continue the conversation with the 2nd message.
        messages.append(message_2)
        response = generate_conversation(
            bedrock_client, model_id, system_prompts, messages)

        output_message = response['output']['message']
        messages.append(output_message)

        # Show the complete conversation.
        for message in messages:
            print(f"Role: {message['role']}")
            for content in message['content']:
                print(f"Text: {content['text']}")
            print()

    except ClientError as err:
        message = err.response['Error']['Message']
        logging.error("A client error occurred: %s", message)
        print(f"A client error occured: {message}")

    else:
        print(
            f"Finished generating text with model {model_id}.")


if __name__ == "__main__":
    main()

## Call Jurassic 2
You can also call Jurassic-2 family models using the Bedrock Invoke API

In [ ]:
import boto3
import json

bedrock_runtime_client = boto3.client(service_name='bedrock-runtime')

body = json.dumps({
    "prompt": "Translate to spanish: 'Amazon Bedrock is the easiest way to build and scale generative AI applications with base models (FMs)'.", 
    "maxTokens": 200,
    "temperature": 0.5,
    "topP": 0.5
})

modelId = 'ai21.j2-ultra-v1'
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime_client.invoke_model(
    body=body, 
    modelId=modelId, 
    accept=accept, 
    contentType=contentType
)

response_body = json.loads(response.get('body').read())

# text
print(response_body.get('completions')[0].get('data').get('text'))